In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
waypoint_distances = {}
waypoint_durations = {}
all_waypoints = set()

waypoint_data = pd.read_csv("my-waypoints-dist-dur.tsv", sep="\t")

for i, row in waypoint_data.iterrows():
    waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
    waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
    all_waypoints.update([row.waypoint1, row.waypoint2])

In [4]:
home = 'Gronnegata 11, Oslo'

In [17]:
def compute_fitness(solution):
    """
        This function returns the total distance traveled on the current road trip.

        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """

    solution_fitness = 0.0

    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]

    return solution_fitness


def generate_random_agent():
    """
        Creates a random road trip from the waypoints.
    """

    new_random_agent = list(all_waypoints)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)


def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.

        A point mutation swaps the order of two waypoints in the road trip.
    """

    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)

    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]

    return tuple(agent_genome)


def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.

        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """

    agent_genome = list(agent_genome)

    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)

    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]

    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]

    return tuple(agent_genome)


def generate_random_population(pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """

    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent())
    return random_population


def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.

        `generations` and `population_size` must be a multiple of 10.
    """

    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)

    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(population_size)
    # print(population)

    # For `generations` number of repetitions...
    for generation in range(generations):

        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(agent_genome)

        # Take the top 10% shortest road trips and produce offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):

            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                print(agent_genome)
                print("")

            # Create 1 exact copy of each of the top road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-3 point mutations
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, 3))

            # Create 7 offspring with a single shuffle mutation
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population

In [32]:
run_genetic_algorithm(generations=5000, population_size=100)

[('Kongeveien 5, Oslo', 'Nobels gate 32, Oslo', 'Bygdoeynesveien 39, Oslo', 'Bygdoy, Oslo, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Museumsveien 10, Oslo', 'Huk Aveny 35, Oslo', 'nan, Oslo', 'Universitetsgaten 13 (city centre), Oslo'), ('Universitetsgaten 13 (city centre), Oslo', 'Huk Aveny 35, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Bygdoy, Oslo, Oslo', 'nan, Oslo', 'Museumsveien 10, Oslo', 'Kongeveien 5, Oslo', 'Nobels gate 32, Oslo', 'Bygdoeynesveien 39, Oslo'), ('Universitetsgaten 13 (city centre), Oslo', 'Bygdoeynesveien 39, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'nan, Oslo', 'Bygdoy, Oslo, Oslo', 'Nobels gate 32, Oslo', 'Huk Aveny 35, Oslo', 'Kongeveien 5, Oslo', 'Museumsveien 10, Oslo'), ('nan, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'Huk Aveny 35, Oslo', 'Bygdoeynesveien 39, Oslo', 'Museumsveien 10, Oslo', 'Nobels gate 32, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo'), ('nan, Oslo', 'Universitetsgaten 13 (city centre), Oslo', '

In [19]:
generations = 5000
population_size = 100

population_subset_size = int(population_size / 10.)
generations_10pct = int(generations / 10.)
# print(population_subset_size) -> 10
# print(generations_10pct) -> 500

# Create a random population of `population_size` number of solutions.
# population = generate_random_population(population_size)

random_population = []
for agent in range(population_size):
    
    # Creates a random road trip from the waypoints.
    # random_population.append(generate_random_agent())
    
    new_random_agent = list(all_waypoints)    
    random.shuffle(new_random_agent)
    # print(new_random_agent)
    
    # return tuple(new_random_agent)
    random_population.append(tuple(new_random_agent))
    
# return random_population
population = random_population

In [25]:
# for generation in range(generations):
# Taking the first iteration
generation = 0

# Compute the fitness of the entire current population
population_fitness = {}

for agent_genome in population:
    if agent_genome in population_fitness:
        continue

    # population_fitness[agent_genome] = compute_fitness(agent_genome)
    
    # def compute_fitness(solution):
    # This function returns the total distance traveled on the current road trip.

    # The genetic algorithm will favor road trips that have shorter
    # total distances traveled.
    solution = agent_genome

    solution_fitness = 0.0

    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        # print(solution_fitness)
        
    # print(solution_fitness)
    # print()
    
    population_fitness[agent_genome] = solution_fitness
    print(population_fitness[agent_genome])

43428.0
34897.0
33081.0
38965.0
42048.0
48463.0
44196.0
41693.0
53051.0
48463.0
42491.0
48127.0
39833.0
42490.0
30685.0
42845.0
38083.0
38862.0
49099.0
40070.0
38863.0
39187.0
47879.0
49346.0
38861.0
39937.0
38445.0
45018.0
39267.0
43619.0
48216.0
45018.0
43427.0
43515.0
42791.0
43618.0
48375.0
32261.0
41798.0
33888.0
47110.0
39937.0
49558.0
38368.0
49583.0
43515.0
38472.0
48215.0
39055.0
48454.0
48454.0
35370.0
44914.0
52782.0
37784.0
34765.0
41112.0
41940.0
38911.0
31606.0
48453.0
34834.0
33660.0
43621.0
37545.0
43428.0
42553.0
39267.0
39044.0
36793.0
53136.0
44001.0
32555.0
33328.0
38091.0
38391.0
53161.0
37587.0
32531.0
28872.0
42694.0
38169.0
33881.0
49258.0
38368.0
41917.0
38114.0
32618.0
44884.0
44778.0
44222.0
48375.0
33691.0
53029.0
49479.0
27111.0
36767.0
35475.0
52781.0
43429.0


In [31]:
# The best trips: we get new paths starting from these
best_trips = sorted(population_fitness, key=population_fitness.get)[:population_subset_size]
for i in best_trips:
    print(population_fitness[i])

27111.0
28872.0
30685.0
31606.0
32261.0
32531.0
32555.0
32618.0
33081.0
33328.0


In [35]:
# Take the top 10% shortest road trips and produce offspring each from them
new_population = []
for rank, agent_genome in enumerate(sorted(population_fitness,
                                           key=population_fitness.get)[:population_subset_size]):

    if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
        print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                               population_fitness[agent_genome],
                                                               len(population_fitness)))
        print(agent_genome)
        print("")

    # Create 1 exact copy of each of the top road trips
    new_population.append(agent_genome)

    # Create 2 offspring with 1-3 point mutations
    print("Mutate agent:")
    for offspring in range(2):
        # new_population.append(mutate_agent(agent_genome, 3))
        # def mutate_agent(agent_genome, max_mutations=3):
        # Applies 1 - `max_mutations` point mutations to the given road trip.
        # A point mutation swaps the order of two waypoints in the road trip.
        max_mutations = 3

        agent_genome = list(agent_genome)
        print(agent_genome)
        num_mutations = random.randint(1, max_mutations)

        for mutation in range(num_mutations):
            swap_index1 = random.randint(0, len(agent_genome) - 1)
            swap_index2 = swap_index1

            while swap_index1 == swap_index2:
                swap_index2 = random.randint(0, len(agent_genome) - 1)

            agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
        
        new_population.append(tuple(agent_genome))
        print(tuple(agent_genome))
        print()


    # Create 7 offspring with a single shuffle mutation
    print("Shuffle_mutation:")
    for offspring in range(7):
        # new_population.append(shuffle_mutation(agent_genome))
        # def shuffle_mutation(agent_genome):
        # Applies a single shuffle mutation to the given road trip.
        # A shuffle mutation takes a random sub-section of the road trip
        # and moves it to another location in the road trip.
    
        agent_genome = list(agent_genome)
        print(agent_genome)

        start_index = random.randint(0, len(agent_genome) - 1)
        length = random.randint(2, 20)

        genome_subset = agent_genome[start_index:start_index + length]
        agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]

        insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
        agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]

        new_population.append(tuple(agent_genome))
        print(tuple(agent_genome))
        print()

# Replace the old population with the new population of offspring
for i in range(len(population))[::-1]:
    del population[i]

population = new_population

Generation 0 best: 27111 | Unique genomes: 100
('Huk Aveny 35, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Nobels gate 32, Oslo', 'Kongeveien 5, Oslo', 'Bygdoy, Oslo, Oslo', 'Museumsveien 10, Oslo', 'Bygdoeynesveien 39, Oslo')

Mutate agent:
['Huk Aveny 35, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Nobels gate 32, Oslo', 'Kongeveien 5, Oslo', 'Bygdoy, Oslo, Oslo', 'Museumsveien 10, Oslo', 'Bygdoeynesveien 39, Oslo']
('Museumsveien 10, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo', 'Huk Aveny 35, Oslo', 'Nobels gate 32, Oslo')

['Museumsveien 10, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Universitetsgaten 13 (city centre), Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Bygdoy, Oslo, Oslo', 'Kongeveien 5, Oslo', 'Huk Aveny 35, Oslo', 'Nobels gate 32, Oslo']
(